In [7]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [9]:
path = '/gdrive/My Drive/ML:March2020/data/'
data= pd.read_csv(path+'paint_mnist.csv',header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
data.shape

(3599, 785)

In [11]:
cols=data.shape[1]
print("columns= {}".format (cols))

columns= 785


In [12]:
y=np.array(data.iloc[:,:1])
x=np.array(data.iloc[:,1:cols])

In [13]:
y.shape, x.shape


((3599, 1), (3599, 784))

In [14]:
b=np.unique(y)

In [15]:
classes= b.size
classes

4

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=2)
x_train.shape, x_test.shape,y_train.shape,y_test.shape

((2699, 784), (900, 784), (2699, 1), (900, 1))

In [18]:
x=np.array(x_train)
y=np.array(y_train)
x.shape, y.shape

((2699, 784), (2699, 1))

In [19]:
rows=x.shape[0]
rows

2699

In [20]:
x=np.insert(x,0,values=np.ones(rows),axis=1)
x.shape

(2699, 785)

In [21]:
params=x.shape[1]
all_theta=np.zeros((classes, params))
all_theta.shape

(4, 785)

In [22]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [23]:
sigmoid(0),sigmoid(10),sigmoid(-100)

(0.5, 0.9999546021312976, 3.7200759760208356e-44)

In [24]:
def computeCost(theta,x,y):
    theta=np.matrix(theta)
    x=np.matrix(x)
    y=np.matrix(y)
    first=np.multiply(-y,np.log(sigmoid(x*theta.T)))
    second=np.multiply((1-y),np.log(1-sigmoid(x*theta.T)))
    cost= np.sum(first-second)
    cost= cost/(2*len(x))
    return cost


In [25]:
cost= computeCost(all_theta,x,y)
print("cost={}".format(cost))

cost=1.3862943611198906


In [26]:
learning_rate=1
def gradient(theta,x,y):
    theta=np.matrix(theta)
    x=np.matrix(x)
    y=np.matrix(y)
    error=sigmoid(x*theta.T)-y
    grad=((x.T*error)/len(x))
    grad=grad*learning_rate
    return grad

In [27]:
from scipy.optimize import minimize
def one_vs_all(x,y,classes,new_theta):
  params=x.shape[1]
  rows=x.shape[0]
  for i in range(0,classes):
      theta=new_theta[i,:]
      y_i=np.array([1 if label ==i else 0 for label in y])
      y_i=np.reshape(y_i,(rows,1))
      fmin=minimize(fun=computeCost, x0=theta, args=(x,y_i), method='TNC',jac=gradient)
      new_theta[i,:]=fmin.x
  return new_theta


In [28]:
new_theta=one_vs_all(x,y,classes,all_theta)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


In [29]:
computeCost(new_theta,x,y)

19.215211942542247

In [30]:
def predict_all(x,theta):
    x=np.matrix(x)
    theta=np.matrix(theta)
    h=sigmoid(x*theta.T)
    h_argmax= np.argmax(h,axis=1)
    return h_argmax,h

In [31]:
y_pred,h=predict_all(x,new_theta)
print(y_pred)

[[2]
 [3]
 [1]
 ...
 [0]
 [1]
 [1]]


In [32]:
def acc(y_pred,y):
  correct=[1 if a==b else 0 for (a,b) in zip(y_pred,y)]
  accuracy=(sum(correct))/(len(correct))
  return accuracy

In [33]:
accuracy=acc(y_pred,y)
print ("Training data accuracy={}%".format(accuracy*100))

Training data accuracy=97.55464987032234%


In [34]:
t_x=np.array(x_test)
t_y=np.array(y_test)

In [35]:
t_rows=t_x.shape[0]
t_x=np.insert(t_x,0,values=np.ones(t_rows),axis=1)

In [36]:
t_y_pred,h=predict_all(t_x,new_theta)
print("Test data accuracy={}%".format(acc(t_y_pred,t_y)*100))

Test data accuracy=75.55555555555556%


In [37]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display
Im=Image.open('/gdrive/My Drive/ML:March2020/Participants/Sasidev/Untitled1.png')
display(Im)

In [38]:
Im=Im.convert('L')
Im=ImageOps.invert(Im)
display(Im)

In [39]:
Im=np.array(Im)
Im=np.ravel(Im)
Im.shape

(784,)

In [40]:
Im=Im.reshape(1,-1)

In [41]:
input_x=np.insert(Im,0,1,axis=1)
input_x.shape

(1, 785)

In [42]:
pred,h=predict_all(input_x,new_theta)
print("confidence matrix={}".format(h))
print('predicted class is:{}'.format(pred.item()))
print("confidence={}%".format (((h[0,pred])*100).item()))

confidence matrix=[[6.36689951e-14 2.70473209e-04 9.99999628e-01 3.26348400e-01]]
predicted class is:2
confidence=99.99996284353728%
